In [3]:
from utils.preprocessing import preprocessingV1
from utils.modelization import saveModel,loadModel,submitModel

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn import preprocessing

In [27]:

from sklearn.feature_selection import f_classif
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import SelectPercentile

from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import RepeatedStratifiedKFold

from sklearn.pipeline import Pipeline

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler

from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier, GradientBoostingClassifier, StackingClassifier, VotingClassifier
#from xgboost.sklearn import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

from sklearn.metrics import f1_score


from sklearn.metrics import classification_report, ConfusionMatrixDisplay
from scipy.stats import uniform, randint


In [9]:
df_train = pd.read_csv("datasets/train_radiomics_hipocamp.csv")
df_test = pd.read_csv("datasets/test_radiomics_hipocamp.csv")

X_full_train, y_full_train = preprocessingV1(df_train)

X_test = preprocessingV1(df_test,False)

In [39]:

from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X_full_train,y_full_train,test_size=0.1,stratify=y_full_train, random_state=1234)

results = {}
models = {}

# Bagging with Decision tree

In [51]:
sss = StratifiedShuffleSplit(n_splits=10, test_size= 20, random_state= 2022)
dt_model = DecisionTreeClassifier(max_depth=2,random_state=2022)
bg_model = BaggingClassifier(estimator= dt_model, bootstrap= True)

grid_p = { 
    'n_estimators' : [20,40,60,80,100],
    'estimator__max_depth' : [2,4,6] 
    }

grid_bg = GridSearchCV(estimator= bg_model, param_grid= grid_p, cv = sss,refit=True,return_train_score=True, scoring= "f1_macro", n_jobs=-1)
grid_bg.fit(X_train,y_train)

bst_bg_model = grid_bg.best_estimator_
bst_bg_score = bst_bg_model.score(X_val,y_val)

y_preds = bst_bg_model.predict(X_val)
macro_f1 = f1_score(y_val, y_preds, average='macro')

results['Bagg'] = macro_f1
models['Bagg'] = bst_bg_model


print("Train macro F1-Score", grid_bg.best_score_)
print("Val Macro F1-Score:", macro_f1)

print(bst_bg_model)
print(grid_bg.best_params_)

bst_bg_predictions = bst_bg_model.predict(X_val)
print(classification_report(y_val,bst_bg_predictions, zero_division=1))

Train macro F1-Score 0.3525085731262202
Val Macro F1-Score: 0.35238095238095235
BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=4,
                                                   random_state=2022),
                  n_estimators=60)
{'estimator__max_depth': 4, 'n_estimators': 60}
              precision    recall  f1-score   support

       AD-AD       0.75      0.50      0.60         6
       CN-CN       0.73      0.80      0.76        10
      CN-MCI       1.00      0.00      0.00         1
      MCI-AD       0.25      0.29      0.27         7
     MCI-MCI       0.12      0.14      0.13         7

    accuracy                           0.45        31
   macro avg       0.57      0.35      0.35        31
weighted avg       0.50      0.45      0.45        31



In [49]:



# Train macro F1-Score 0.3525085731262202
# Val Macro F1-Score: 0.35238095238095235
# BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=4,
#                                                   random_state=2022)

0.31883053221288515

In [103]:
print(grid_bg.best_score_)

#saveModel(grid_bg.best_estimator_,"cv_bagging_dt_mf1_3525")
#submitModel(grid_bg.best_estimator_.predict(X_test),'test_predictions_bagging_dt_mf1_03525')

#Train macro F1-Score 0.3525085731262202
#Val Macro F1-Score: 0.35238095238095235
#BaggingClassifier(estimator=DecisionTreeClassifier(max_depth=4,
 #                                                  random_state=2022),
 #                 n_estimators=60)
#{'estimator__max_depth': 4, 'n_estimators': 60}

0.3525085731262202


# Bagging with Random Forest

In [75]:
rf_model = RandomForestClassifier(bootstrap= True, max_depth= 2, verbose= 1, random_state=123)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=123)

grid_p = { 
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10,20],
    'min_samples_leaf': [1, 2, 4,8],
    'max_features': ['sqrt', 'log2']
    }

grid_rf = GridSearchCV(estimator= rf_model, 
                       param_grid= grid_p,
                       cv = cv,
                       refit=True,return_train_score=True, 
                       scoring= "f1_macro", n_jobs=-1, verbose= 1)
grid_rf.fit(X_full_train,y_full_train)

results['Bagg_rf'] =  grid_rf.best_score_
models['Bagg_rf'] =  grid_rf.best_estimator_

print('Best Mean Macro F1: %.3f' %  grid_rf.best_score_)
print('Best Config: %s' % grid_rf.best_params_)

print(grid_rf.best_score_)

Fitting 30 folds for each of 216 candidates, totalling 6480 fits


c:\Users\gusta\.conda\envs\mypython3v\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,
[Parallel(n_jobs=1)]: Done  49 tasks      | elapsed:    0.1s


Best Mean Macro F1: 0.326
Best Config: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}
0.32618107236807714


In [99]:

saveModel(grid_rf.best_estimator_,"cv_bagging_rf_mf1_0326")



# Best Mean Macro F1: 0.326
# Best Config: {'max_depth': 10, 'max_features': 'sqrt', 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}

# Gradient boosting

In [86]:


gbc_model = GradientBoostingClassifier( verbose= 1, random_state=123)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=123)


param_distrib = {
    'n_estimators': randint(50, 500),           # Number of boosting stages
    'learning_rate': uniform(0.01, 0.3),       # Step size shrinkage
    'max_depth': randint(3, 100),               # Maximum depth of each tree
    'min_samples_split': randint(2, 20),       # Minimum samples to split an internal node
    'min_samples_leaf': randint(1, 10),        # Minimum samples required at a leaf node
    'subsample': uniform(0.6, 0.4),            # Fraction of samples used per tree
    'max_features': ['sqrt', 'log2', None]     # Features considered for the best split
}

grid_gbc = RandomizedSearchCV(estimator= gbc_model, 
                       param_distributions= param_distrib,
                       n_iter=10,
                       cv = cv,
                       refit=True,return_train_score=True, 
                       scoring= "f1_macro", n_jobs=-1, verbose= 1, random_state=123)

grid_gbc.fit(X_full_train,y_full_train)

results['Bagg_gbc'] =  grid_gbc.best_score_
models['Bagg_gbc'] =  grid_gbc.best_estimator_

print('Best Mean Macro F1: %.3f' %  grid_gbc.best_score_)
print('Best Config: %s' % grid_gbc.best_params_)

print(grid_gbc.best_score_)

Fitting 30 folds for each of 10 candidates, totalling 300 fits
      Iter       Train Loss      OOB Improve   Remaining Time 
         1           1.2062           0.0976            3.65m
         2           1.0418           0.1662            3.96m
         3           0.9094           0.0511            4.07m
         4           0.7982           0.2378            4.13m
         5           0.7010          -0.0151            4.18m
         6           0.6171           0.1233            4.17m
         7           0.5467           0.1387            4.18m
         8           0.4882           0.1458            4.20m
         9           0.4280           0.0690            4.18m
        10           0.3746          -0.0014            4.18m
        20           0.1247           0.0224            4.13m
        30           0.0450          -0.0009            4.03m
        40           0.0157          -0.0005            3.90m
        50           0.0060           0.0013            3.75m
      

In [98]:
saveModel(grid_gbc.best_estimator_,"cv_gbc_mf1_0339")
# Best Config: {'learning_rate': 0.10690382853418266, 'max_depth': 6, 'max_features': None, 'min_samples_leaf': 8, 'min_samples_split': 14, 'n_estimators': 296, 'subsample': 0.9503681331729179}


# Xgboost

In [96]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
xgb_model = XGBClassifier( verbose= 1, random_state=123)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=2, random_state=123)

param_distributions = {
    'n_estimators': randint(50, 200),  # Number of boosting rounds (trees)
    'learning_rate': uniform(0.1, 0.3),  # Step size shrinking, in the range [0.01, 0.3]
    'max_depth': randint(3, 15),  # Maximum depth of the decision tree
    'min_child_weight': randint(1, 15),  # Minimum sum of instance weight (hessian) for a child
    'subsample': uniform(0.5, 0.5),  # Fraction of samples to use for fitting (0.5 to 1)
    'colsample_bytree': uniform(0.5, 0.5),  # Fraction of features to use for each tree
    'colsample_bylevel': uniform(0.5, 0.5),  # Fraction of features to use for each level
    'gamma': uniform(0, 0.5),  # Minimum loss reduction to make a further partition
    'reg_alpha': uniform(0, 1),  # L1 regularization term on weights
    'reg_lambda': uniform(0, 1),  # L2 regularization term on weights
    'objective': ['multi:softmax'],  # Objective function (binary or multi-class)
}

# RandomizedSearchCV 
rs_xgb = RandomizedSearchCV(estimator= xgb_model, 
                       param_distributions= param_distributions,
                       n_iter=2,
                       cv = cv,
                       refit=True,return_train_score=True, 
                       scoring= "f1_macro", n_jobs=-1, verbose= 1, random_state=123)

rs_xgb.fit(X_full_train, label_encoder.fit_transform(y_full_train) )

results['rs_xgb'] =  rs_xgb.best_score_
models['rs_xgb'] =  rs_xgb.best_estimator_

print('Best Mean Macro F1: %.3f' %  rs_xgb.best_score_)
print('Best Config: %s' % rs_xgb.best_params_)

print(rs_xgb.best_score_)

Fitting 30 folds for each of 2 candidates, totalling 60 fits


c:\Users\gusta\.conda\envs\mypython3v\Lib\site-packages\xgboost\core.py:158: UserWarning: [21:50:41] WARNING: D:\bld\xgboost-split_1732667012888\work\src\learner.cc:740: 
Parameters: { "scale_pos_weight", "verbose" } are not used.

  warnings.warn(smsg, UserWarning)


Best Mean Macro F1: 0.346
Best Config: {'colsample_bylevel': 0.7005087783306018, 'colsample_bytree': 0.8136585046688083, 'gamma': 0.16207544668977053, 'learning_rate': 0.17342778308617596, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 118, 'objective': 'multi:softmax', 'reg_alpha': 0.6317920176870504, 'reg_lambda': 0.44025717806407627, 'scale_pos_weight': 1.8372648456358434, 'subsample': 0.8561650906943812}
0.3455182303834074


In [108]:
# Best Mean Macro F1: 0.346
# Best Config: {'colsample_bylevel': 0.7005087783306018, 'colsample_bytree': 0.8136585046688083, 'gamma': 0.16207544668977053, 'learning_rate': 0.17342778308617596, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 118, 'objective': 'multi:softmax', 'reg_alpha': 0.6317920176870504, 'reg_lambda': 0.44025717806407627, 'scale_pos_weight': 1.8372648456358434, 'subsample': 0.8561650906943812}


print(label_encoder.fit_transform(y_full_train))
print(y_full_train)

[1 1 0 2 1 3 1 0 4 4 1 0 0 0 3 3 1 4 3 3 0 1 3 4 1 1 0 1 1 1 1 3 4 3 1 0 2
 1 0 3 1 3 0 1 0 3 0 0 4 4 3 0 1 1 4 0 4 1 4 1 1 4 4 1 2 1 4 0 4 0 4 2 0 4
 1 3 1 1 1 3 0 0 1 4 1 3 0 4 1 4 0 4 0 1 2 4 4 4 3 3 3 0 4 3 3 4 0 4 4 4 0
 4 1 3 1 4 4 0 1 4 3 0 3 1 4 4 1 1 1 0 3 1 0 3 4 3 1 1 4 3 4 4 0 0 1 0 2 0
 4 0 1 1 3 3 4 3 1 4 0 3 1 3 1 4 0 0 1 4 4 3 4 3 2 1 0 3 1 3 1 4 1 4 4 4 3
 1 1 1 3 1 1 0 0 1 3 0 3 3 0 1 1 1 1 0 3 1 3 3 4 3 1 0 1 1 4 0 1 1 0 4 0 0
 3 2 1 4 3 1 1 1 3 4 1 4 3 2 3 3 3 3 0 1 3 0 1 1 3 3 4 1 1 0 1 0 4 4 0 3 3
 3 4 4 1 0 4 0 1 3 1 4 1 1 4 4 1 1 3 1 3 3 1 4 0 3 4 4 1 3 0 3 0 4 0 1 1 3
 4 1 4 2 1 1 3 4 1]
0        CN-CN
1        CN-CN
2        AD-AD
3       CN-MCI
4        CN-CN
        ...   
300      CN-CN
301      CN-CN
302     MCI-AD
303    MCI-MCI
304      CN-CN
Name: Transition, Length: 305, dtype: object


In [105]:

#saveModel(rs_xgb.best_estimator_,"cv_xgboost_mf1_0346")

#saveModel(grid_bg.best_estimator_,"cv_bagging_dt_mf1_3525")
submitModel(  label_encoder.inverse_transform(rs_xgb.best_estimator_.predict(X_test)),'test_predictions_xgboost_mf1_0346')


#Best Mean Macro F1: 0.346
#Best Config: {'colsample_bylevel': 0.7005087783306018, 'colsample_bytree': 0.8136585046688083, 'gamma': 0.16207544668977053, 'learning_rate': 0.17342778308617596, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 118, 'objective': 'multi:softmax', 'reg_alpha': 0.6317920176870504, 'reg_lambda': 0.44025717806407627, 'scale_pos_weight': 1.8372648456358434, 'subsample': 0.8561650906943812}

In [22]:
best_pipeline = search.best_estimator_
y_test_results = best_pipeline.predict(X_test)


In [23]:
#Best Mean Macro F1: 0.328
#Best Config: {'scaler': MinMaxScaler(), 'svc__C': 100, 'svc__degree': 1, 'svc__gamma': 'auto', 'svc__kernel': 'poly'}
#saveModel(best_pipeline,"svc_predictions_svc_mf1_0328_poly_submission_022811")



#Best Mean Macro F1: 0.347
#Best Config: {'scaler': MinMaxScaler(), 'svc__C': 1000, 'svc__coef0': 1.0, 'svc__degree': 1, 'svc__gamma': 'scale', 'svc__kernel': 'sigmoid'}
#saveModel(best_pipeline,"svc_mf1_0347_sigmoid_submission_028741")

# loadedm = loadModel("best_models/lr_anova_local_0354_submission_039047.pkl")